In [7]:
#!pip install langchain
#!pip install openai
#!pip install dotenv
#!pip install httpx
#!pip install opencc
#!pip install ipywidgets
%load_ext autoreload 
%autoreload 2
import json
from pprint import pprint
import os
import sys
if "../src/" not in sys.path:
    sys.path.append("../src/lc_tools/")
    sys.path.append("../src/")

from dotenv import load_dotenv
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain, SimpleSequentialChain, SequentialChain, TransformChain
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import openai

from lope_tools import SenseTagTool, QuerySenseFromDefinitionTool, QuerySenseFromLemmaTool, QuerySenseFromExamplesTool

load_dotenv('../.env', override=True)
openai.api_key = os.environ['OPENAI_API_KEY']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load LLM

In [4]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0,
    client=None
)

# Tools
Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents. Tools take in a single string as input and return a string as output.

More info about tools: https://python.langchain.com/en/latest/modules/agents/tools/getting_started.html


* `SenseTagTool(text)`: Tokenizes and tags `text` using `DistilTagger`.
* `QuerySenseFromDefinitionTool(text)`: Returns all senses that contain `text` in its definition. `text` can be a regular expression.
* `QuerySenseFromLemmaTool(text)`: Returns all senses that contain `text` in its lemma. `text` can be a regular expression.
* `QuerySenseFromExampleTool(text)`: Returns all senses that contain `text` in its example. `text` can be a regular expression.

In [5]:
# Example
pprint(SenseTagTool().run("他打算打我一頓。"))

('([詞] 他 || [詞性] Nh || [詞意] [05238501] 代指自己和對方以外的第三人。 (0.9708))\n'
 '([詞] 打算 || [詞性] VF || [詞意] [06603401] 初步決定要做後述事件。 (1.0000))\n'
 '([詞] 打 || [詞性] VC || [詞意] [05229102] 用手或手持物打後述對象，使其感到疼痛或受到傷害。 (0.9984))\n'
 '([詞] 我 || [詞性] Nh || [詞意] [05238701] 代指說話者。 (0.9666))\n'
 '([詞] 一 || [詞性] Neu || [詞意] [06727301] 事物的全體。 (0.5460))\n'
 '([詞] 頓 || [詞性] Nf || [詞意] [06001902] 計算負面經驗事件次數的單位。 (1.0000))\n'
 '([詞] 。 || [詞性] PERIODCATEGORY || [詞意] )')


In [11]:
pprint(list(json.loads(QuerySenseFromDefinitionTool().run("電腦")).items())[:3])

[('電通所',
  {'all_examples': ['目前電信總局已結合電信研究所及工研院<電通所>，成立VOD推動小組。',
                    '中心已於上週向<電通所>訂購一套單機版的「中華民國期刊論文索引光碟系統」，並擬於近期內安裝上線使用。',
                    '由於外界疑慮日後得標廠商，將形成壟斷市場情況，因此，<電通所>在採購規格訂定上，已著手進行有效防堵。'],
   'definition': '工業技術研究院，研究電腦與通訊工業的機構，在西元1990年成立，已於西元2006年更名為資訊與通訊工業研究所。',
   'head_word': '電通所'}),
 ('命令',
  {'all_examples': ['如果你不確定<命令>語法，可從編輯器叫出<命令>語法參照表進行查詢。',
                    '本程式是以連續式<命令>的方式訓練神經網路上的車子訊號如何行走的\r\n程式。',
                    '以後只要利用telnet<命令>即可進行遠程連線，<命令>格式如英文書目方法步驟2中所示。',
                    '其<命令>格式為：vi．cshrc仔細查看檔中是否包含下令三行<命令>，若無則必須自行加入後回存。'],
   'definition': '人給電腦的指令。',
   'head_word': '命令'}),
 ('寫成',
  {'all_examples': ['spice的原始版本是以Fortran<寫成>，通稱為2g6。',
                    '此性能指標以C語言<寫成>，為一主要以CPU整數運算為主的benchmark程式。',
                    '請問ADO.NET的資料可以轉換為ADO 的Recordset嗎？我想<寫成>COM元件，讓舊的ASP可以使用。',
                    '在DOS '
                    '中,你可能會從事一些例行的重覆性工作,此時你會將這些重覆性的命令<寫成>批次檔,只要執行這個批次檔就等於執行這些命令。'],
   'definition': '以電腦可判讀的文字符號組成程式或程式碼。

In [13]:
pprint(json.loads(QuerySenseFromLemmaTool().run("電腦")))

{'個人電腦': {'all_examples': ['一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。',
                           '當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。',
                           '面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。'],
          'definition': '提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。',
          'head_word': '個人電腦'},
 '電腦': {'all_examples': ['小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。'],
        'definition': '比喻計算或記憶能力很強的人。',
        'head_word': '電腦'}}


In [14]:
pprint(list(json.loads(QuerySenseFromExamplesTool().run("電腦")).items())[:3])

[('罷',
  {'all_examples': ['我們呼籲全港市民主持公道，<罷>食麥當勞，直至麥當勞改善工資水平為止。',
                    '三百多位群眾演員昨天<罷>演《功夫足球》，李先生無奈地說，這一切都是盒飯惹的禍。',
                    '鍾麗緹於拍攝當日忽然改變裸露尺度，更<罷>拍部分「露股」甫士，令拍攝氣氛一度緊張。',
                    '中國是東芝在亞洲最重要的市場，現受事件影響，四川、東北等地已經有部分電腦專賣店公開表示<罷>賣東芝產品。'],
   'definition': '因為特定原因而停止某種活動，通常帶有抗議性質。',
   'head_word': '罷'}),
 ('抱',
  {'all_examples': ['每天下課回到家，還<抱>著電話講個不停。',
                    '有些人平時不上課，到考前才<抱>著課本猛背。',
                    '不要整天躲宿舍<抱>電腦，有時間應該出去曬曬太陽。'],
   'definition': '花費大量時間於特定事物上。',
   'head_word': '抱'}),
 ('比不上',
  {'all_examples': ['媽媽的手藝很好，大廚師也<比不上>她。',
                    '難道我們的情誼還<比不上>初識的陌生人？',
                    '電腦螢幕的設計如何符合人體工學，都<比不上>正確的操作姿勢，來得重要。'],
   'definition': '沒有達到比較對象的標準。',
   'head_word': '比不上'})]


# Chains
## Why do we need chains?
> Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

In [11]:
# Use SenseTagTool in a chain
# https://python.langchain.com/en/latest/modules/chains/generic/transformation.html
tag_chain = TransformChain(
    input_variables=["original_text"], 
    output_variables=["tagged_text"],  # output_variables are needed for chains further down the line
    transform=lambda inputs: {"tagged_text": SenseTagTool().run(inputs["original_text"])},
)

In [27]:
find_sense_template = """請回答關於斷詞和標註詞意後句子的問題。保留原始的解釋。使用簡潔有力的言詞回答。

原句：{original_text}
斷詞和標註後的句子：
{tagged_text}
問題：「{target_token}」在原句出現哪些意思？請使用編號列表回答。最後加上\n。

"""

# Create chat prompt with system and human messages
system_message = SystemMessage(content="You are a helpful assistant that specializes in Taiwan Mandarin linguistics.")
human_message_prompt = HumanMessagePromptTemplate.from_template(find_sense_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

# Output key is used by the next chain to get the output of this chain
find_sense_chain = LLMChain(prompt=chat_prompt, llm=llm, output_key="senses")

In [15]:
# Call the chain by itself
original_text = "我好想他但是我想他沒想我。"
tagged_text = SenseTagTool().run(original_text)
target_token = "想"
out = find_sense_chain.run(
    original_text=original_text,
    tagged_text=tagged_text,
    target_token=target_token,
)

「想」在原句中出現了兩個意思：
1. [05047208] 因分開而思念後述對象。
2. [05047204] 在心中對特定事件提出推測性的論斷。

In [29]:
# Creating sentences from senses
sentence_from_sense_template = """請以簡潔有力的言詞根據以下提供的詞意回答問題。

詞意：
{senses}
根據每個詞意造三個句子。請使用編號列表回答。造句前要重述詞意。

"""
system_message = SystemMessage(content="You are a helpful assistant that specializes in Taiwan Mandarin linguistics.")
human_message_prompt = HumanMessagePromptTemplate.from_template(sentence_from_sense_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])

sentence_from_sense_chain = LLMChain(prompt=chat_prompt, llm=llm)

In [30]:
# SequentialChain for multiple input/outputs and multiple chains
# https://python.langchain.com/en/latest/modules/chains/generic/sequential_chains.html
overall_chain = SequentialChain(
    chains=[tag_chain, find_sense_chain, sentence_from_sense_chain],
    input_variables=["original_text", "target_token"],
    output_variables=["tagged_text", "senses"],
    verbose=True,
)
original_text = "我好想他但是我想他沒想我。"
tagged_text = SenseTagTool().run(original_text)
target_token = "想"
out = overall_chain(
    {
        "original_text": original_text,
        "target_token": target_token,
    }
)




> Entering new SequentialChain chain...
「想」在原句中出現以下兩個意思：
1. [05047208] 因分開而思念後述對象。
2. [05047204] 在心中對特定事件提出推測性的論斷。1. 「想」的詞意是因分開而思念後述對象。
- 我們分開已經一個月了，我很想你。
- 每當我看到這個城市的夜景，就會想起你。
- 離開台灣後，我一直想念台灣的美食。

2. 「想」的詞意是在心中對特定事件提出推測性的論斷。
- 我想這個計畫應該可以成功。
- 他的表現很差，我想他可能需要更多的訓練。
- 這個問題很複雜，我想我需要更多的時間來思考。
> Finished chain.


# Chat and Memory
> Memory involves keeping a concept of state around throughout a user’s interactions with an language model. A user’s interactions with a language model are captured in the concept of ChatMessages, so this boils down to ingesting, capturing, transforming and extracting knowledge from a sequence of chat messages.

More info: 
* https://python.langchain.com/en/latest/modules/memory/getting_started.html
* https://python.langchain.com/en/latest/modules/memory/examples/adding_memory.html

In [28]:
doc = QuerySenseFromLemmaTool().run("電腦")
pprint(json.loads(doc))

{'個人電腦': {'all_examples': ['一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。',
                           '當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。',
                           '面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。'],
          'definition': '提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。',
          'head_word': '個人電腦'},
 '電腦': {'all_examples': ['小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。'],
        'definition': '比喻計算或記憶能力很強的人。',
        'head_word': '電腦'}}


In [29]:
template = """You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0,
    client=None
)

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],  # exclude partial_variables from input_variables
    partial_variables={"context": doc},  # use partial_variables to only fill in part of the template
    template=template,
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
conversation = LLMChain(llm=llm, memory=memory, prompt=prompt, verbose=True)


In [30]:
conversation.predict(human_input="總共有幾個詞義？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦"}}


Human: 總共有幾個詞義？
Chatbot:
這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。
> Finished chain.


'這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。'

In [32]:
conversation.predict(human_input="第一個詞意有幾個例句？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦"}}

Human: 總共有幾個詞義？
AI: 這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。
Human: 第一個詞意有幾個例句？
Chatbot:
「電腦」這個詞義有一個例句。
> Finished chain.


'「電腦」這個詞義有一個例句。'

In [33]:
conversation.predict(human_input="另一個有幾個例句？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦"}}

Human: 總共有幾個詞義？
AI: 這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。
Human: 第一個詞意有幾個例句？
AI: 「電腦」這個詞義有一個例句。
Human: 另一個有幾個例句？
Chatbot:
「個人電腦」這個詞義有三個例句。
> Finished chain.


'「個人電腦」這個詞義有三個例句。'

In [34]:
conversation.predict(human_input="總結它最後例句在說什麼？")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot specializing in Taiwan Mandarin having a conversation with a human.

以下是來自漢語詞彙網的一些詞義的 JSON 格式資訊。每個頂層鍵都是該詞義的詞頭。根據這些資訊和提出的問題，請回答問題。

{"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦"}}

Human: 總共有幾個詞義？
AI: 這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。
Human: 第一個詞意有幾個例句？
AI: 「電腦」這個詞義有一個例句。
Human: 另一個有幾個例句？
AI: 「個人電腦」這個詞義有三個例句。
Human: 總結它最後例句在說什麼？
Chatbot:
最後一個例句提到了名豐資訊推出的

'最後一個例句提到了名豐資訊推出的「天眼多工版」，這是一套能在DOS系統下做到一機遙控多機功能的程式，讓使用者可以同時在前景模式下修改程式，背景模式下傳輸檔案更新程式，提高了使用者的效率。'

In [35]:
# Get Chat history
memory.chat_memory.messages

[HumanMessage(content='總共有幾個詞義？', additional_kwargs={}),
 AIMessage(content='這個 JSON 格式資訊中總共有兩個詞義。分別是「電腦」和「個人電腦」。', additional_kwargs={}),
 HumanMessage(content='第一個詞意有幾個例句？', additional_kwargs={}),
 AIMessage(content='「電腦」這個詞義有一個例句。', additional_kwargs={}),
 HumanMessage(content='另一個有幾個例句？', additional_kwargs={}),
 AIMessage(content='「個人電腦」這個詞義有三個例句。', additional_kwargs={}),
 HumanMessage(content='總結它最後例句在說什麼？', additional_kwargs={}),
 AIMessage(content='最後一個例句提到了名豐資訊推出的「天眼多工版」，這是一套能在DOS系統下做到一機遙控多機功能的程式，讓使用者可以同時在前景模式下修改程式，背景模式下傳輸檔案更新程式，提高了使用者的效率。', additional_kwargs={})]

# Conversation Agent with Tools
> Other agents are often optimized for using tools to figure out the best response, which is not ideal in a conversational setting where you may want the agent to be able to chat with the user as well. This is accomplished with a specific type of agent (conversational-react-description) which expects to be used with a memory component.

More info: https://python.langchain.com/en/latest/modules/agents/agents/examples/conversational_agent.html

In [38]:
tools = [
    SenseTagTool(),
    QuerySenseFromDefinitionTool(),
    QuerySenseFromLemmaTool(),
    QuerySenseFromExamplesTool(),
]

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0,
    client=None
)

memory = ConversationBufferMemory(memory_key="chat_history")
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

In [39]:
agent_chain.run("「電腦」出現在lemma有哪些詞意？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: QuerySensesFromLemma
Action Input: 電腦Thought: Do I need to use a tool? Yes
Action: QuerySensesFromLemma
Action Input: 電腦
Observation: {"電腦": {"all_examples": ["小<電腦>是三年級時，大家取的，因為我都替他們解決問題，所以大家就叫我小<電腦>。"], "definition": "比喻計算或記憶能力很強的人。", "head_word": "電腦"}, "個人電腦": {"all_examples": ["一般電腦用戶最關心的系統，莫過於<個人電腦>上的多媒體產品，坦白說，<個人電腦>的多媒體系統，雖然相關周邊或有推出，但因<個人電腦>目前微處理器運算速度太慢、記憶體太小，仍無法順利運作，或許未來會有所改進。", "當然，網威亦想透過本案獲得新市場，網威目前在ＰＣ級網路作業系統上擁有不可動搖的地位，但來自ＭＳ—ＷｉｎｄｏｗｓＮＴ、ＩＢＭＬＡＮＳｅｒｖｅｒ的壓力與日俱增，該公司當然也希望透過本交易，讓Ｎｅｔｗａｒｅ與Ｕｎｉｘ更密切結合，甚至形成一套整合<個人電腦>、網路、工作站的Ｕｎｉｘ作業系統。", "面對<個人電腦>走向多工作業時代，名豐資訊於日前領先推出一套能在ＤＯＳ系統下做到一機遙控多機功能的「天眼多工版」，讓使用者可一面在前景模式下修改本程式，一面在背景模式下傳輸檔案更新程式，不用浪費寶貴的時間。"], "definition": "提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。", "head_word": "個人電腦"}}
Thought:Do I need to use a tool? No
AI: 「電腦」在lemma中有兩個詞意：「電腦」的詞義是「比喻計算或記憶能力很強的人。」，「個人電腦」的詞義是「提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。」。Do I need t

'「電腦」在lemma中有兩個詞意：「電腦」的詞義是「比喻計算或記憶能力很強的人。」，「個人電腦」的詞義是「提供個人使用的一種資料處理裝置，能自動接受並儲存、處理輸入的資料，然後經由一組預先存放在機器內的指令逐步引導下產生輸出結果。」。'

# Chain types 
> * `stuff`: uses ALL of the text from the documents in the prompt
> * `map_reduce`: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.
> * `refine`:  It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches
> * `map-rerank`: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

More info: 
* https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a